In [151]:
from itertools import combinations as comb, permutations as perm

n = 3 # no. of inputs
m = 2 # no. of outputs

a = [i for i in range(n)]
b = [i for i in range(n, 2*n)]
c = [[i + j for j in range(n)] for i in range(2*n, 2*n + n^2, 3)]

# Compute functions from {0, 1, 2} -> {0, 1}
F = [[int(c) for c in bin(i)[2:].zfill(n)] for i in range(2^n)]
V = []
Vs = []
Vsym = []


# Map each input choice to an index from 0 to n^2 - 1
A = {(i, j): n*i + j for i in range(n) for j in range(n)}
# Map each output choice to an index from 0 to m^2 - 1
B = {(i, j): m*i + j for i in range(m) for j in range(m)}

# Create the vertices of the classical polytope

num_functions = len(F)

# Alice's function
for i in range(num_functions):
    # Bob's function
    for j in range(num_functions):
        vertex = [QQ(0)] * (m^2*n^2)
        # For every input pair, Alice and Bob sample a function
        f_A, f_B = F[i], F[j]
        for x_A, x_B in A.keys():
            # Compute their outputs depending on the function they chose
            y_A, y_B = f_A[x_A], f_B[x_B]
            vertex[A[(x_A, x_B)] + n^2 * B[(y_A, y_B)]] = 1
            
        # If they picked the same function, we are synchronous
        if i == j:
            Vs.append(vertex)
        
        V.append(vertex)

# Symmetric classical correlations
for i in range(num_functions):
    for j in range(i, num_functions):
        vertex = [QQ(0)] * (m^2*n^2)
    
        for x_A, x_B in A.keys():
            y_A, y_B = F[i][x_A], F[j][x_B]
            vertex[A[(x_A, x_B)] + n^2 * B[(y_A, y_B)]] += 1/2
            y_A, y_B = F[j][x_A], F[i][x_B]
            vertex[A[(x_A, x_B)] + n^2 * B[(y_A, y_B)]] += 1/2
            
        Vsym.append(vertex)

def change_coordinates(V_from):
    V_to = []
    for v in V_from:
        vert = [0] * (2*n + n^2)
        for x_A, x_B in A.keys():
            vert[a[x_A]] = sum( (-1)^y_A * v[A[(x_A, x_B)] + n^2 * B[(y_A, y_B)]] for y_A, y_B in B.keys() )
            vert[b[x_B]] = sum( (-1)^y_B * v[A[(x_A, x_B)] + n^2 * B[(y_A, y_B)]] for y_A, y_B in B.keys() )
            vert[c[x_A][x_B]] = sum( (-1)^(y_A + y_B) * v[A[(x_A, x_B)] + n^2 * B[(y_A, y_B)]] for y_A, y_B in B.keys() )
        V_to.append(vert)
    return V_to

Vertices = change_coordinates(V)
Vertices_Synch = change_coordinates(Vs)
Vertices_Symm = change_coordinates(Vsym)

var_A = [var('a' + str(i)) for i in range(3)]
var_B = [var('b' + str(i)) for i in range(3)]
var_C = [[var('c' + str(i) + str(j)) for j in range(3)] for i in range(3)]

def print_ineq(inequalities):
    for ineq in inequalities:
        #show(ineq)
        ineq_exp = ineq[0]
        for i in range(n):
            if ineq[i + 1] != 0:
                ineq_exp += ineq[i + 1] * var_A[i]
        for i in range(n, 2*n):
            if ineq[i + 1] != 0:
                ineq_exp += ineq[i + 1] * var_B[i - n]
                
        for i in range(2*n, 2*n + n^2):
            if ineq[i + 1] != 0:
                idx = i - 2*n
                j, k = idx // n, idx % n
                ineq_exp += ineq[i + 1] * var_C[j][k]
        ineq_exp = ineq_exp >= 0
        show(ineq_exp)

In [152]:
J0 = lambda v: 1/4 * (1 + v[c[0][1]] - v[c[0][2]] - v[c[1][2]])
J1 = lambda v: 1/4 * (1 - v[c[0][1]] + v[c[0][2]] - v[c[1][2]])
J2 = lambda v: 1/4 * (1 - v[c[0][1]] - v[c[0][2]] + v[c[1][2]])
J3 = lambda v: 1/4 * (1 + v[c[0][1]] + v[c[0][2]] + v[c[1][2]])

In [153]:
def add_condition(cons, indices_and_values):
    eq = [cons] + [0] * (n^2 + 2*n)
    for (idx, val) in indices_and_values:
        eq[idx + 1] = val
    
    return eq

# Add the positivity conditions
positivity = []
for xa in range(n):
    for xb in range(n):
        positivity.append(add_condition(1, [(a[xa], 1), (b[xb], 1), (c[xa][xb], 1)]))
        positivity.append(add_condition(1, [(a[xa], 1), (b[xb], -1), (c[xa][xb], -1)]))
        positivity.append(add_condition(1, [(a[xa], -1), (b[xb], 1), (c[xa][xb], -1)]))
        positivity.append(add_condition(1, [(a[xa], -1), (b[xb], -1), (c[xa][xb], 1)]))
        positivity.append(add_condition(3, [(a[xa], -1), (b[xb], -1), (c[xa][xb], -1)]))
        positivity.append(add_condition(3, [(a[xa], -1), (b[xb], 1), (c[xa][xb], 1)]))
        positivity.append(add_condition(3, [(a[xa], 1), (b[xb], -1), (c[xa][xb], 1)]))
        positivity.append(add_condition(3, [(a[xa], 1), (b[xb], 1), (c[xa][xb], -1)])) 

# Equations for the symmetric polytopes 
symmetric = []
for x in range(n):
    symmetric.append(add_condition(0, [(a[x], 1), (b[x], -1)]))
    
for xa in range(n):
    for xb in range(xa + 1, n):
        symmetric.append(add_condition(0, [(c[xa][xb], 1), (c[xb][xa], -1)]))

# Equations for the synchronous polytopes
synchronicity = []
for x in range(n):
    synchronicity.append(add_condition(0, [(a[x], 1), (b[x], -1)]))
    synchronicity.append(add_condition(1, [(c[x][x], -1)]))

In [154]:
nspoly = Polyhedron(ieqs = positivity)
symnspoly = Polyhedron(ieqs = positivity, eqns = symmetric)
syncnspoly = Polyhedron(ieqs = positivity, eqns = synchronicity)

hvpoly = Polyhedron(vertices = Vertices)
symhvpoly = Polyhedron(vertices = Vertices_Symm)
synchvpoly = Polyhedron(vertices = Vertices_Synch)

In [155]:
print("{:>53s} \t {} \t {} \t {} \t {}".format("Vertex", "J0", "J1", "J2", "J3"))
print("------------------------------------------------------------------------------------")
for v in syncnspoly.vertices_list():
    print("{:>53s} \t {} \t {} \t {} \t {}".format(str(v), J0(v), J1(v), J2(v), J3(v)))

                                               Vertex 	 J0 	 J1 	 J2 	 J3
------------------------------------------------------------------------------------
  [-1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 	 0 	 0 	 0 	 1
[-1, -1, 1, -1, -1, 1, 1, 1, -1, 1, 1, -1, -1, -1, 1] 	 1 	 0 	 0 	 0
     [-1, 0, 0, -1, 0, 0, 1, 0, 0, 0, 1, 1, 0, -1, 1] 	 0 	 0 	 1/2 	 1/2
     [-1, 0, 0, -1, 0, 0, 1, 0, 0, 0, 1, -1, 0, 1, 1] 	 1/2 	 1/2 	 0 	 0
  [-1, 1, 1, -1, 1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1] 	 0 	 0 	 1 	 0
[-1, 1, -1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1] 	 0 	 1 	 0 	 0
   [0, 0, 0, 0, 0, 0, 1, -1, -1, -1, 1, -1, 1, -1, 1] 	 1/2 	 1/2 	 1/2 	 -1/2
      [0, 0, 0, 0, 0, 0, 1, -1, -1, 1, 1, 1, 1, 1, 1] 	 0 	 0 	 1 	 0
    [0, 0, 0, 0, 0, 0, 1, -1, -1, -1, 1, 1, 1, -1, 1] 	 0 	 0 	 1 	 0
   [0, 0, 0, 0, 0, 0, 1, -1, -1, -1, 1, -1, -1, 1, 1] 	 1/2 	 1/2 	 1/2 	 -1/2
[1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, 1, -1, 1, 1] 	 0 	 0 	 1 	 0
     [0, 0, 0, 0, 0, 0, 1, -1, -1, 1, 1, -1, 

In [ ]:
print("Synchronous hidden variables polytope")
print_ineq(synchvpoly.inequalities())